### Testing Multimodal Models via the IONOS OpenAI-Compatible API

IONOS provides OpenAI-compatible inference endpoints.

We load credentials from a `.env` file and initialize an OpenAI client
with a custom base URL pointing to the IONOS infrastructure.

This allows reuse of the same multimodal code
(text + image input) across providers.

In [5]:
import os

# DWD proxy
os.environ["HTTP_PROXY"]  = "http://ofsquid.dwd.de:8080"
os.environ["HTTPS_PROXY"] = "http://ofsquid.dwd.de:8080"

# Optional but recommended
os.environ["http_proxy"]  = os.environ["HTTP_PROXY"]
os.environ["https_proxy"] = os.environ["HTTPS_PROXY"]

In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load variables from .env
load_dotenv()

IONOS_BASE_URL = "https://openai.inference.de-txl.ionos.com/v1"

# Initialize OpenAI-compatible client for IONOS
client_ionos = OpenAI(
    api_key=os.getenv("IONOS_API_KEY"),
    base_url=IONOS_BASE_URL
)

print("IONOS client initialized.")
print("IONOS API key loaded:", bool(os.getenv("IONOS_API_KEY")))


IONOS client initialized.
IONOS API key loaded: True


### Listing Models Available on the IONOS Inference Endpoint

Before testing multimodal capabilities, we first query the IONOS
OpenAI-compatible API to list all models available to this account.

This mirrors the OpenAI workflow and allows us to select
appropriate candidates for multimodal testing.

In [7]:
# Query available models from the IONOS endpoint
models_ionos = client_ionos.models.list()

print("Available IONOS models:\n")
for m in models_ionos.data:
    print(" -", m.id)

Available IONOS models:

 - meta-llama/Meta-Llama-3.1-8B-Instruct
 - meta-llama/Meta-Llama-3.1-405B-Instruct-FP8
 - meta-llama/Llama-3.3-70B-Instruct
 - mistralai/Mistral-Nemo-Instruct-2407
 - openGPT-X/Teuken-7B-instruct-commercial
 - mistralai/Mistral-Small-24B-Instruct
 - openai/gpt-oss-120b
 - BAAI/bge-m3
 - sentence-transformers/paraphrase-multilingual-mpnet-base-v2
 - black-forest-labs/FLUX.1-schnell
 - stabilityai/stable-diffusion-xl-base-1.0
 - BAAI/bge-large-en-v1.5
 - meta-llama/CodeLlama-13b-Instruct-hf


In [16]:
from PIL import Image
import io
import base64

def load_and_compress_image(
    path,
    max_size=(512, 512),
    format="JPEG",
    quality=70,
):
    """
    Load an image, resize it, compress it, and return base64 encoding.

    - max_size: maximum (width, height)
    - format: 'JPEG' or 'PNG'
    - quality: JPEG quality (1–95)
    """
    img = Image.open(path).convert("RGB")

    # Resize while keeping aspect ratio
    img.thumbnail(max_size)

    # Write to in-memory buffer
    buf = io.BytesIO()
    img.save(buf, format=format, quality=quality, optimize=True)
    buf.seek(0)

    # Base64 encode
    return base64.b64encode(buf.read()).decode("utf-8")

In [18]:
image_b64  = load_and_compress_image(
    "radar_map_germany.png",
    max_size=(512, 512),
    quality=70,
)

image2_b64 = load_and_compress_image(
    "cth_map.png",
    max_size=(512, 512),
    quality=50,
)

print("Radar image size (base64 chars):", len(image_b64))
print("CTH image size (base64 chars):", len(image2_b64))

Radar image size (base64 chars): 26468
CTH image size (base64 chars): 40076


In [20]:
def test_model_ionos(model_name, image_b64):
    """
    Test whether an IONOS-hosted model supports image input.
    """
    try:
        response = client_ionos.chat.completions.create(
            model=model_name,
            messages=[{
                "role": "user",
                "content": [
                    {"type": "text",
                     "text": "Describe this image briefly."},
                    {"type": "image_url",
                     "image_url": {
                         "url": f"data:image/png;base64,{image_b64}"
                     }}
                ]
            }],
            max_tokens=500,
        )

        print(f"\n[IONOS | {model_name}]")
        print(response.choices[0].message.content)

    except Exception as e:
        print(f"\n[IONOS | {model_name}] FAILED:", e)


In [21]:
test_model_ionos("mistralai/Mistral-Small-24B-Instruct",image_b64)


[IONOS | mistralai/Mistral-Small-24B-Instruct]
The image is a radar reflectivity map from the German Weather Service (DWD). It shows radar reflectivity data overlaid on a map of Germany with state borders outlined. The radar reflectivity is color-coded, with a color bar on the right side indicating the reflectivity values in decibels per square meter (dBZ).

- The color bar ranges from dark blue to red, representing reflectivity values from 0 to 40 dBZ.
- Dark blue represents lower reflectivity values, indicating lighter precipitation or no precipitation.
- Green and yellow indicate moderate reflectivity values, suggesting moderate precipitation.
- Orange and red indicate higher reflectivity values, suggesting heavy precipitation.

The map shows areas of varying reflectivity, with the most significant precipitation appearing in the northern and central parts of Germany. The precipitation areas are scattered, with some regions showing higher reflectivity values, indicating heavier rain

In [22]:
test_model_ionos("mistralai/Mistral-Small-24B-Instruct",image2_b64)


[IONOS | mistralai/Mistral-Small-24B-Instruct]
The image is a map depicting the cloud top height (CTH) over Central Europe. The map uses a color gradient to represent different cloud top heights in meters, ranging from 2500 meters to 22500 meters. The color scale on the right side of the image indicates the corresponding heights, with lighter colors representing higher cloud tops and darker colors representing lower cloud tops.

Key observations from the map include:

1. **High Clouds**: Areas with lighter colors, such as yellow and light green, indicate higher cloud tops, which are around 22500 meters. These areas are scattered but not predominant.

2. **Mid-Level Clouds**: Regions with moderate colors, such as green and blue, indicate mid-level cloud tops, ranging from approximately 10000 meters to 17500 meters. These areas are more widespread.

3. **Low Clouds**: Darker colors, such as dark blue and purple, represent lower cloud tops, ranging from 2500 meters to 7500 meters. These 